In [ ]:
import ray
import random 
import os 
import psutil
random.seed(42)

In [39]:
@ray.remote
class Accumulator:
    def __init__(self):
        self.value = 0

    def add(self, x):
        self.value += sum(x)
        return self.value

In [49]:
@ray.remote
def fn_add(x):
    pid = os.getpid()
    return sum(x), psutil.Process(pid).cpu_num()

In [50]:
'''
Note: This gives you number of GPUs, not cores per GPU.
You typically allocate per-GPU because Ray assigns tasks/actors per GPU.
'''
ray.cluster_resources().get('GPU', 0), ray.cluster_resources().get('CPU', 1)

(1.0, 16.0)

In [51]:
arr = [random.randint(0, 100) for _ in range(10000000)]
SPLITS = int(ray.cluster_resources().get('CPU', 1))
n = len(arr)

In [58]:
%%time
lb, op = 0, []
while lb < n:
    ub = min(lb + n // SPLITS, n)
    op.append(fn_add.remote(arr[lb:ub]))
    lb = ub
# Get the results from all remote calls
results = ray.get(op)
print('CPU cores:' + str(set(cpu for add, cpu in results)))
# The sum is the sum of all accumulator results
total = sum([add for add, cpu in results])
print('Total sum:', total)

CPU cores:{2, 8, 10, 12, 14}
Total sum: 500065011
CPU times: user 345 ms, sys: 39.7 ms, total: 385 ms
Wall time: 343 ms


In [59]:
%%time 
print('Total sum:', sum(arr))

Total sum: 500065011
CPU times: user 26.4 ms, sys: 0 ns, total: 26.4 ms
Wall time: 26.2 ms


## Task

**You cannot control sum, that why the advantage is not showing.**
**To show the advantage, we simulate a task which takes 1s.**


In [64]:
import time

def task_s():
    time.sleep(1)

@ray.remote
def task():
    time.sleep(1)

In [65]:
%%time 
# Sequential 
JOBS = 10
for _ in range(JOBS):
    task_s()

CPU times: user 34.2 ms, sys: 12.7 ms, total: 46.9 ms
Wall time: 10 s


In [67]:
%%time 
# Parallel 
JOBS = 10
op = [task.remote() for _ in range(JOBS)]
result = ray.get(op)

CPU times: user 12.1 ms, sys: 4.27 ms, total: 16.4 ms
Wall time: 1.01 s
